# 🔄 步骤 3：概念去重与关系合并

这个 notebook 负责：
1. 使用语义嵌入识别相似概念
2. 合并重复概念
3. 更新关系中的概念引用
4. 过滤低质量概念
5. 生成最终的知识图谱数据

**⏱️ 预计时间**：2-5 分钟（取决于概念数量）

## 1. 加载环境和配置

In [ ]:
import os
import sys
import json
import yaml
import logging
from typing import Dict, List, Tuple
from datetime import datetime

import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# 加载环境变量
if os.path.exists('/content/PWD_Project/.env'):
    with open('/content/PWD_Project/.env', 'r') as f:
        for line in f:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                os.environ[key] = value

PROJECT_ROOT = os.environ.get('PROJECT_ROOT', '/content/PWD_Project')
CONFIG_PATH = os.environ.get('CONFIG_PATH', f'{PROJECT_ROOT}/config/config_colab.yaml')

os.chdir(PROJECT_ROOT)

# 加载配置
with open(CONFIG_PATH, 'r', encoding='utf-8') as f:
    CONFIG = yaml.safe_load(f)

# 配置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger('Deduplicator')

print(f"✅ 配置已加载")
print(f"   相似度阈值: {CONFIG['deduplication']['similarity_threshold']}")
print(f"   嵌入模型: {CONFIG['deduplication']['embedding_model']}")

## 2. 加载上一步的提取结果

In [ ]:
output_dir = CONFIG['output']['base_directory']

# 加载概念
concepts_file = f"{output_dir}/concepts_raw.csv"
if not os.path.exists(concepts_file):
    print(f"❌ 找不到概念文件: {concepts_file}")
    print("   请先运行 02_概念提取.ipynb")
    raise FileNotFoundError(concepts_file)

concepts_df = pd.read_csv(concepts_file)
print(f"✅ 加载了 {len(concepts_df)} 个原始概念")

# 加载关系
relationships_file = f"{output_dir}/relationships_raw.csv"
if os.path.exists(relationships_file):
    relationships_df = pd.read_csv(relationships_file)
    print(f"✅ 加载了 {len(relationships_df)} 个原始关系")
else:
    relationships_df = pd.DataFrame()
    print("⚠️  未找到关系文件")

## 3. 概念去重器

In [ ]:
class ConceptDeduplicator:
    """使用语义嵌入进行概念去重"""
    
    def __init__(self, model_name: str, similarity_threshold: float = 0.85):
        self.model_name = model_name
        self.similarity_threshold = similarity_threshold
        
        logger.info(f"加载嵌入模型: {model_name}")
        self.model = SentenceTransformer(model_name)
        logger.info("模型加载完成")
    
    def deduplicate_concepts(self, concepts_df: pd.DataFrame) -> Tuple[pd.DataFrame, Dict]:
        """去重概念并返回映射关系"""
        if concepts_df.empty:
            return concepts_df, {}
        
        # 获取唯一概念
        unique_concepts = concepts_df['entity'].unique().tolist()
        logger.info(f"唯一概念数: {len(unique_concepts)}")
        
        # 计算嵌入
        logger.info("计算概念嵌入...")
        embeddings = self.model.encode(unique_concepts, show_progress_bar=True)
        
        # 计算相似度矩阵
        logger.info("计算相似度矩阵...")
        similarity_matrix = cosine_similarity(embeddings)
        
        # 找到相似概念并建立映射
        logger.info("识别相似概念...")
        concept_mapping = {}  # 旧概念 -> 规范概念
        used_indices = set()
        
        for i in range(len(unique_concepts)):
            if i in used_indices:
                continue
            
            # 选择第一个作为规范概念
            canonical = unique_concepts[i]
            concept_mapping[canonical] = canonical
            
            # 查找相似的概念
            for j in range(i + 1, len(unique_concepts)):
                if j in used_indices:
                    continue
                
                if similarity_matrix[i][j] >= self.similarity_threshold:
                    # 映射到规范概念
                    concept_mapping[unique_concepts[j]] = canonical
                    used_indices.add(j)
        
        logger.info(f"识别出 {len(used_indices)} 个重复概念")
        
        # 应用映射
        concepts_df['entity'] = concepts_df['entity'].map(
            lambda x: concept_mapping.get(x, x)
        )
        
        # 聚合重复概念的属性
        deduplicated_df = concepts_df.groupby('entity').agg({
            'category': 'first',
            'importance': 'max',  # 取最高重要性
            'chunk_id': 'first',
            'source_pdf': 'first'
        }).reset_index()
        
        logger.info(f"去重后: {len(unique_concepts)} -> {len(deduplicated_df)} 个概念")
        
        return deduplicated_df, concept_mapping

print("✅ ConceptDeduplicator 类定义完成")

## 4. 执行概念去重

In [ ]:
print("\n" + "="*60)
print("开始概念去重")
print("="*60 + "\n")

start_time = datetime.now()

# 创建去重器
deduplicator = ConceptDeduplicator(
    model_name=CONFIG['deduplication']['embedding_model'],
    similarity_threshold=CONFIG['deduplication']['similarity_threshold']
)

# 执行去重
deduplicated_concepts, concept_mapping = deduplicator.deduplicate_concepts(concepts_df)

end_time = datetime.now()
duration = end_time - start_time

print("\n" + "="*60)
print("概念去重完成")
print("="*60)
print(f"\n耗时: {duration}")
print(f"原始概念: {len(concepts_df['entity'].unique())}")
print(f"去重后: {len(deduplicated_concepts)}")
print(f"去除重复: {len(concepts_df['entity'].unique()) - len(deduplicated_concepts)}")

## 5. 更新关系中的概念引用

In [ ]:
if not relationships_df.empty:
    print("\n更新关系中的概念引用...")
    
    # 应用概念映射到关系
    relationships_df['node_1'] = relationships_df['node_1'].map(
        lambda x: concept_mapping.get(x, x)
    )
    relationships_df['node_2'] = relationships_df['node_2'].map(
        lambda x: concept_mapping.get(x, x)
    )
    
    # 移除自环（概念指向自己）
    before_count = len(relationships_df)
    relationships_df = relationships_df[relationships_df['node_1'] != relationships_df['node_2']]
    removed_self_loops = before_count - len(relationships_df)
    
    # 移除重复关系（保留权重最高的）
    relationships_df = relationships_df.sort_values('weight', ascending=False)
    relationships_df = relationships_df.drop_duplicates(
        subset=['node_1', 'node_2', 'edge'], 
        keep='first'
    )
    
    print(f"✅ 关系更新完成")
    print(f"   移除自环: {removed_self_loops}")
    print(f"   最终关系数: {len(relationships_df)}")
else:
    print("⚠️  没有关系需要更新")

## 6. 过滤低质量概念

In [ ]:
print("\n" + "="*60)
print("过滤低质量概念")
print("="*60 + "\n")

min_importance = CONFIG['filtering']['min_importance']
min_connections = CONFIG['filtering']['min_connections']

# 计算每个概念的连接数
if not relationships_df.empty:
    concept_connections = pd.concat([
        relationships_df['node_1'].value_counts(),
        relationships_df['node_2'].value_counts()
    ], axis=1).fillna(0).sum(axis=1)
    
    deduplicated_concepts['connections'] = deduplicated_concepts['entity'].map(
        lambda x: concept_connections.get(x, 0)
    ).astype(int)
else:
    deduplicated_concepts['connections'] = 0

# 应用过滤
before_filter = len(deduplicated_concepts)

filtered_concepts = deduplicated_concepts[
    (deduplicated_concepts['importance'] >= min_importance) |
    (deduplicated_concepts['connections'] >= min_connections)
]

print(f"过滤前: {before_filter} 个概念")
print(f"过滤后: {len(filtered_concepts)} 个概念")
print(f"移除: {before_filter - len(filtered_concepts)} 个低质量概念")

# 过滤关系（只保留有效概念之间的关系）
if not relationships_df.empty:
    valid_concepts = set(filtered_concepts['entity'].unique())
    
    before_filter_rel = len(relationships_df)
    filtered_relationships = relationships_df[
        (relationships_df['node_1'].isin(valid_concepts)) &
        (relationships_df['node_2'].isin(valid_concepts))
    ]
    
    print(f"\n关系过滤:")
    print(f"   过滤前: {before_filter_rel}")
    print(f"   过滤后: {len(filtered_relationships)}")
else:
    filtered_relationships = relationships_df

## 7. 查看最终结果

In [ ]:
print("\n📊 最终概念样例（按重要性排序）:")
print(filtered_concepts.nlargest(15, 'importance')[['entity', 'category', 'importance', 'connections']])

print("\n📊 概念类别分布:")
print(filtered_concepts['category'].value_counts())

print("\n📊 重要性分布:")
print(filtered_concepts['importance'].value_counts().sort_index())

In [ ]:
if not filtered_relationships.empty:
    print("\n📊 最终关系样例（按权重排序）:")
    print(filtered_relationships.nlargest(15, 'weight')[['node_1', 'node_2', 'edge', 'weight']])
    
    print("\n📊 关系类型分布:")
    print(filtered_relationships['edge'].value_counts().head(10))

## 8. 保存最终结果

In [ ]:
# 保存去重后的概念
final_concepts_file = f"{output_dir}/concepts_final.csv"
filtered_concepts.to_csv(final_concepts_file, index=False, encoding='utf-8-sig')
print(f"✅ 最终概念已保存: {final_concepts_file}")

# 保存去重后的关系
if not filtered_relationships.empty:
    final_relationships_file = f"{output_dir}/relationships_final.csv"
    filtered_relationships.to_csv(final_relationships_file, index=False, encoding='utf-8-sig')
    print(f"✅ 最终关系已保存: {final_relationships_file}")

# 保存概念映射（用于追溯）
mapping_file = f"{output_dir}/concept_mapping.json"
with open(mapping_file, 'w', encoding='utf-8') as f:
    json.dump(concept_mapping, f, ensure_ascii=False, indent=2)
print(f"✅ 概念映射已保存: {mapping_file}")

# 保存统计信息
dedup_stats = {
    'timestamp': datetime.now().isoformat(),
    'duration': str(duration),
    'original_concepts': len(concepts_df['entity'].unique()),
    'deduplicated_concepts': len(deduplicated_concepts),
    'filtered_concepts': len(filtered_concepts),
    'duplicates_removed': len(concepts_df['entity'].unique()) - len(deduplicated_concepts),
    'low_quality_removed': len(deduplicated_concepts) - len(filtered_concepts),
    'original_relationships': len(relationships_df) if not relationships_df.empty else 0,
    'final_relationships': len(filtered_relationships),
    'similarity_threshold': CONFIG['deduplication']['similarity_threshold'],
    'min_importance': min_importance,
    'min_connections': min_connections
}

stats_file = f"{output_dir}/deduplication_stats.json"
with open(stats_file, 'w', encoding='utf-8') as f:
    json.dump(dedup_stats, f, ensure_ascii=False, indent=2)
print(f"✅ 统计信息已保存: {stats_file}")

## 9. 去重完成

### ✅ 完成情况

- 原始概念数量 → 去重后数量
- 移除的重复概念数量
- 过滤的低质量概念数量
- 最终知识图谱规模

### 📝 下一步

请打开 **`04_Neo4j导入.ipynb`** 将知识图谱导入数据库

### 💡 提示

- 最终数据已准备好导入 Neo4j
- 可以调整相似度阈值重新去重
- 概念映射文件可用于追溯原始概念

In [ ]:
# 显示完成总结
print("\n" + "="*60)
print("🎉 概念去重步骤完成！")
print("="*60)
print(f"\n📊 处理结果:")
print(f"   - 原始概念: {dedup_stats['original_concepts']}")
print(f"   - 去除重复: {dedup_stats['duplicates_removed']}")
print(f"   - 去除低质量: {dedup_stats['low_quality_removed']}")
print(f"   - 最终概念: {dedup_stats['filtered_concepts']}")
print(f"   - 最终关系: {dedup_stats['final_relationships']}")
print(f"   - 耗时: {dedup_stats['duration']}")
print(f"\n📁 文件保存在: {output_dir}")
print(f"\n➡️  下一步: 打开 04_Neo4j导入.ipynb")